# Step 1: Experiment in a notebook
In this step you run data processing and model training and evaluation in the notebook locally. You don't use `sagemaker` or `boto3` packages.

![](img/six-steps-1.png)

In [5]:
!aws codeartifact login --tool pip --domain cirrus-ml-ds-domain --domain-owner 813736554012 --repository cirrus-ml-ds-shared-repo

Successfully configured pip to use AWS CodeArtifact repository https://cirrus-ml-ds-domain-813736554012.d.codeartifact.eu-north-1.amazonaws.com/pypi/cirrus-ml-ds-shared-repo/ 
Login expires in 12 hours at 2023-03-07 21:08:00+00:00


In [6]:
# We use the opensource xgboost algorithm to implement the model
# We upgrade the sagemaker library to the latest version, as the built-in Studio image might contain not the latest version of the SageMaker Python SDK
%pip install -q xgboost --upgrade sagemaker 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
docker-compose 1.29.2 requires PyYAML<6,>=3.10, but you have pyyaml 6.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import numpy as np 
import json
import joblib
import xgboost as xgb
import sagemaker
import boto3
import os
from time import gmtime, strftime, sleep
from sklearn.metrics import roc_auc_score
from sagemaker.experiments.run import Run, load_run

sagemaker.__version__

'2.132.0'

## Get env variables & Networking

In [8]:
def get_environment(ssm_params):
    sm = boto3.client("sagemaker")
    ssm = boto3.client("ssm")
    
    list_profiles_resp = sm.list_user_profiles(
        MaxResults=100
    )
    
    domain_id = ""
    for profile in list_profiles_resp['UserProfiles']:
        profile_name = profile['UserProfileName']
        domain_id = profile['DomainId']
        break

    r = sm.describe_domain(
            DomainId=domain_id
        )
    del r["ResponseMetadata"]
    del r["CreationTime"]
    del r["LastModifiedTime"]
    r = {**r, **r["DefaultUserSettings"]}
    del r["DefaultUserSettings"]

    i = {
        **r,
        **{t["Key"]:t["Value"]
            for t in sm.list_tags(ResourceArn=r["DomainArn"])["Tags"]
            if t["Key"] in ["EnvironmentName", "EnvironmentType"]}
    }

    for p in ssm_params:
        try:
            i[p["VariableName"]] = ssm.get_parameter(Name=f"{i['EnvironmentName']}-{i['EnvironmentType']}-{p['ParameterName']}")["Parameter"]["Value"]
        except:
            i[p["VariableName"]] = ""

    return i

ssm_parameters = [
        {"VariableName": "DataBucketName", "ParameterName": "data-bucket-name"},
        {"VariableName": "ModelBucketName", "ParameterName": "model-bucket-name"},
        {"VariableName": "S3KmsKeyId", "ParameterName": "kms-s3-key-arn"},
        {"VariableName": "EbsKmsKeyArn", "ParameterName": "kms-ebs-key-arn"}
    ]
    
env_data = get_environment(ssm_params=ssm_parameters)

env_data["TrainingRole"] = env_data["ExecutionRole"]
region='eu-north-1'

env_data

{'DomainArn': 'arn:aws:sagemaker:eu-north-1:370702650160:domain/d-tdizim9qnor9',
 'DomainId': 'd-tdizim9qnor9',
 'DomainName': 'mlops-dev-eu-north-1-sagemaker-domain',
 'HomeEfsFileSystemId': 'fs-03fc3d37f8623fea2',
 'Status': 'InService',
 'AuthMode': 'IAM',
 'AppNetworkAccessType': 'VpcOnly',
 'SubnetIds': ['subnet-0724be5e7071e7070', 'subnet-01def51ffe7467c71'],
 'Url': 'https://d-tdizim9qnor9.studio.eu-north-1.sagemaker.aws',
 'VpcId': 'vpc-0459a28f3637e285c',
 'KmsKeyId': 'f4664542-0f2e-42ca-b51f-2bec0ad62278',
 'ExecutionRole': 'arn:aws:iam::370702650160:role/sm-mlops-env-EnvironmentIAM-SageMakerExecutionRole-14AU65MVMBUGO',
 'SecurityGroups': ['sg-041054ee4500f96f6'],
 'JupyterServerAppSettings': {'DefaultResourceSpec': {'SageMakerImageArn': 'arn:aws:sagemaker:eu-north-1:243637512696:image/jupyter-server-3',
   'InstanceType': 'system',
   'LifecycleConfigArn': 'arn:aws:sagemaker:eu-north-1:370702650160:studio-lifecycle-config/install-autoshutdown-server-extension'},
  'Lifecycl

In [9]:
from sagemaker.network import NetworkConfig

network_config = NetworkConfig(
    enable_network_isolation=False,
    security_group_ids=env_data["SecurityGroups"],
    subnets=env_data["SubnetIds"],
    encrypt_inter_container_traffic=True)

In [10]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-start-here notebook   ")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")

Stored variables and their in-db values:
baseline_s3_url               -> 's3://sagemaker-eu-north-1-370702650160/from-idea-
experiment_name               -> 'from-idea-to-prod-experiment-06-08-26-51'
input_s3_url                  -> 's3://sagemaker-eu-north-1-370702650160/from-idea-
output_s3_url                 -> 's3://sagemaker-eu-north-1-370702650160/from-idea-
target_col                    -> 'y'
test_s3_url                   -> 's3://sagemaker-eu-north-1-370702650160/from-idea-
train_s3_url                  -> 's3://sagemaker-eu-north-1-370702650160/from-idea-
training_job_name             -> 'from-idea-to-prod-training-2023-03-06-12-31-54-34
validation_s3_url             -> 's3://sagemaker-eu-north-1-370702650160/from-idea-
+++++++++++++++++++++++++++++++++++++++++++++++++
[ERROR] YOU HAVE TO RUN 00-start-here notebook   
+++++++++++++++++++++++++++++++++++++++++++++++++


In [11]:
target_col = "y"

In [12]:
%store target_col

Stored 'target_col' (str)


In [13]:
!echo $target_col

y


In [14]:
session = sagemaker.Session()
sm = session.sagemaker_client

## Load data

In [15]:
# This cell is tagged with `parameters` tag and will be overwritten if the notebook executed headlessly
file_source = "EFS"
file_name = "bank-additional-full.csv"
input_path = "./data/bank-additional" 
output_path = "./data"

In [14]:
# # If run the notebook as a job, non-interactivel or headlessly, the notebook cannot access the Studio EFS volume, download the dataset from S3 instead
# # See the section "Run the notebook as a SageMaker job" for more details
# if file_source != "EFS":
#     print("file_source is not EFS")
#     session.download_data(
#         path=os.path.join(input_path, ""), 
#         bucket=bucket_name,
#         key_prefix=f"{bucket_prefix}/input/{file_name}"
#     )

In [16]:
df_data = pd.read_csv(os.path.join(input_path, file_name), sep=";")

pd.set_option("display.max_columns", 500)  # View all of the columns
df_data  # show first 5 and last 5 rows of the dataframe

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,334,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,383,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,189,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,442,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


## Create an experiment
You can use [Amazon SageMaker Experiments Python SDK](https://sagemaker.readthedocs.io/en/stable/experiments/index.html) to organize all your model development work and track all model runs as `experiment runs`.

[SageMaker Experiments](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html) automatically track the inputs, parameters, configurations, and results of your iterations as `runs`.

Experiments are organized in `runs` and runs organized in `run groups`:

- `Experiment`: A collection of runs that are grouped together. An experiment includes runs for multiple types that can be initiated from anywhere using the SageMaker Python SDK.
- `Run`: Each execution step of a model training process. A run consists of all the inputs, parameters, configurations, and results for one iteration of model training. Custom parameters and metrics can be logged using the `log_parameter`, `log_parameters`, and `log_metric` functions. Custom input and output can be logged using the `log_file` function.

In [17]:
experiment_name = f"from-idea-to-prod-experiment-{strftime('%d-%H-%M-%S', gmtime())}"
experiment_name

'from-idea-to-prod-experiment-07-09-08-33'

In [18]:
%store experiment_name

Stored 'experiment_name' (str)


## Feature engineering

As an example, the processing script implements the following feature engineering:
1. Create a new column called `no_previous_contact`. Set value to `1` when `pdays` is `999` and `0` otherwise
1. Generate a new column to show whether the customer is working based on `job` column
1. Remove the economic features and from the dataset as they would need to be forecasted with high precision to be used as features during inference time
1. Remove `duration` as it is not know before a call is performed
1. Convert categorical variables to numeric using **one hot encoding**
1. Move the target column `y` to the front

In real world you implement additional processing, data quality handling, and feature engineering. You also go via multiple "try & fail" iterations.

In [19]:
# Indicator variable to capture when pdays takes a value of 999
df_data["no_previous_contact"] = np.where(df_data["pdays"] == 999, 1, 0)

# Indicator for individuals not actively employed
df_data["not_working"] = np.where(
    np.in1d(df_data["job"], ["student", "retired", "unemployed"]), 1, 0
)

# remove unnecessary data
df_model_data = df_data.drop(
    ["duration", "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"],
    axis=1,
)

df_model_data = pd.get_dummies(df_model_data)  # Convert categorical variables to sets of indicators

# Replace "y_no" and "y_yes" with a single label column, and bring it to the front:
df_model_data = pd.concat(
    [
        df_model_data["y_yes"].rename(target_col),
        df_model_data.drop(["y_no", "y_yes"], axis=1),
    ],
    axis=1,
)

In [20]:
df_model_data

,y,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,marital_unknown,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,default_no,default_unknown,default_yes,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,0,56,1,999,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
1,0,57,1,999,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
2,0,37,1,999,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
3,0,40,1,999,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
4,0,56,1,999,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,1,73,1,999,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
41184,0,46,1,999,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
41185,0,56,2,999,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
41186,1,44,1,999,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0


## Split data

[SageMaker XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html#InputOutput-XGBoost) expects data in the libSVM or CSV formats, with:

- The target variable in the first column, and
- No header row

In [21]:
# Shuffle and splitting dataset
train_data, validation_data, test_data = np.split(
    df_model_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(df_model_data)), int(0.9 * len(df_model_data))],
)

print(f"Data split > train:{train_data.shape} | validation:{validation_data.shape} | test:{test_data.shape}")

Data split > train:(28831, 60) | validation:(8238, 60) | test:(4119, 60)


In [22]:
# Save data to Studio filesystem
train_data.to_csv(os.path.join(output_path, "train.csv"), index=False, header=False)
validation_data.to_csv(os.path.join(output_path, "validation.csv"), index=False, header=False)
test_data.to_csv(os.path.join(output_path, "test.csv"), index=False, header=False)

## Model training and validation

In [23]:
train_features = train_data.drop(target_col, axis=1)
train_label = pd.DataFrame(train_data[target_col])

In [24]:
# DMatrix is an internal data structure that is used by XGBoost, which
# is optimized for both memory efficiency and training speed. 
# You can construct DMatrix from multiple different sources of data.

dtrain = xgb.DMatrix(train_features, label=train_label)

In [25]:
hyperparams = {
                "max_depth": 5,
                "eta": 0.5,
                "alpha": 2.5,
                "objective": "binary:logistic",
                "subsample" : 0.8,
                "colsample_bytree" : 0.8,
                "min_child_weight" : 3
              }

num_boost_round = 150
nfold = 3
early_stopping_rounds = 10

In [26]:
# Cross-validate on training data
cv_results = xgb.cv(
    params=hyperparams,
    dtrain=dtrain,
    num_boost_round=num_boost_round,
    nfold=nfold,
    early_stopping_rounds=early_stopping_rounds,
    metrics=["auc"],
    seed=10,
)

First, train the model on `nfold` number of folds of the training dataset and run a cross-validation.

In [27]:
cv_results

,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.706917,0.015293,0.704481,0.020513
1,0.740870,0.011272,0.739477,0.012814
2,0.755776,0.004207,0.752108,0.009109
3,0.764206,0.009395,0.759917,0.004238
4,0.771115,0.006108,0.763120,0.007303
5,0.777235,0.005779,0.767532,0.008868
6,0.783116,0.008545,0.768878,0.009694
7,0.785492,0.008955,0.768914,0.010426
8,0.788565,0.007161,0.769469,0.009047
9,0.792847,0.005032,0.770529,0.009175


In [28]:
metrics_data = {
    "binary_classification_metrics": {
        "validation:auc": {
            "value": cv_results.iloc[-1]["test-auc-mean"],
            "standard_deviation": cv_results.iloc[-1]["test-auc-std"]
        },
        "train:auc": {
            "value": cv_results.iloc[-1]["train-auc-mean"],
            "standard_deviation": cv_results.iloc[-1]["train-auc-std"]
        },
    }
}

In [29]:
print(f"Cross-validated train-auc:{cv_results.iloc[-1]['train-auc-mean']:.2f}")
print(f"Cross-validated validation-auc:{cv_results.iloc[-1]['test-auc-mean']:.2f}")

Cross-validated train-auc:0.80
Cross-validated validation-auc:0.77


Now retrain the model on the full training dataset instead of splitting the training dataset across a number of folds. Use the test dataset for early stopping.

In [30]:
test_features = test_data.drop(target_col, axis=1)
test_label = pd.DataFrame(test_data[target_col])
dtest = xgb.DMatrix(test_features, label=test_label)

### Create a run
Create a new run using the [`Run`](https://sagemaker.readthedocs.io/en/stable/experiments/sagemaker.experiments.html#run) class and call the [`log_parameters()`](https://sagemaker.readthedocs.io/en/stable/experiments/sagemaker.experiments.html#sagemaker.experiments.Run.log_parameters) and [`log_artifact()`](https://sagemaker.readthedocs.io/en/stable/experiments/sagemaker.experiments.html#sagemaker.experiments.Run.log_artifact) methods to record information to the run.

You can use [`log_file()`](https://sagemaker.readthedocs.io/en/stable/experiments/sagemaker.experiments.html#sagemaker.experiments.Run.log_file) method to upload local files to S3 to persistently store all data for the run.

In [31]:
run_suffix = strftime('%Y-%m-%M-%S', gmtime())
run_suffix

'2023-03-09-30'

In [32]:
with Run(experiment_name=experiment_name,
         run_name=f"feature-engineering-{run_suffix}",
         run_display_name="feature-engineering",
         sagemaker_session=session) as run:
    run.log_parameters(
        {
            "train": 0.7,
            "validate": 0.2,
            "test": 0.1
        }
    )
    # Log input dataset metadata and output
    run.log_artifact(name="marketing-dataset", value="./data/bank-additional/bank-additional-full.csv", media_type="text/csv", is_output=False)
    run.log_artifact(name="train-csv", value="./data/train.csv", media_type="text/csv")
    run.log_artifact(name="validation-csv", value="./data/validation.csv", media_type="text/csv")
    run.log_artifact(name="test-csv", value="./data/test.csv", media_type="text/csv")

#    time.sleep(8) # wait until resource tags are propagated to the run

### Train a model
Use the class[`Run`](https://sagemaker.readthedocs.io/en/stable/experiments/sagemaker.experiments.html#run) to log model metrics.

In [33]:
# in the production code you need to use the unique ids
run_suffix = strftime('%Y-%m-%M-%S', gmtime())
run_suffix

'2023-03-12-54'

In [35]:
# Train the model for different max_depth values
for i, d in enumerate([2, 5, 10, 15, 20]):
    hyperparams["max_depth"] = d
    
    print(f"Fit estimator with max_depth={d}")
    run_name = f"training-{i}-{run_suffix}"
    
    with Run(experiment_name=experiment_name,
             run_name=run_name,
             run_display_name=f"max-depth-{d}",
             sagemaker_session=session) as run:
        # Train the model
        model = (
            xgb.train(
                params=hyperparams, 
                dtrain=dtrain, 
                evals = [(dtrain,'train'), (dtest,'eval')], 
                num_boost_round=num_boost_round, 
                early_stopping_rounds=early_stopping_rounds, 
                verbose_eval = 0
            )
        )

        # Calculate metrics
        test_auc = roc_auc_score(test_label, model.predict(dtest))
        train_auc = roc_auc_score(train_label, model.predict(dtrain))
        
        # Log metrics to the run
        run.log_parameters(hyperparams)
        run.log_metric(name="test_auc", value = test_auc, step=d)
        run.log_metric(name="train_auc", value = train_auc, step=d)
        run.log_metric(name="alpha", value = hyperparams['alpha'], step=d)
        run.log_metric(name="eta", value = hyperparams['eta'], step=d)
        run.log_metric(name="colsample_bytree", value = hyperparams['colsample_bytree'], step=d)
        run.log_metric(name="max_depth", value = hyperparams['max_depth'], step=d)

        # time.sleep(8) # wait until resource tags are propagated to the run

        print(f"Test AUC: {test_auc:.4f} | Train AUC: {train_auc:.4f}")

INFO:sagemaker.experiments.run:The run (training-0-2023-03-12-54) under experiment (from-idea-to-prod-experiment-07-09-08-33) already exists. Loading it. Note: sagemaker.experiments.load_run is recommended to use when the desired run already exists.


Fit estimator with max_depth=2
[09:22:49] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Test AUC: 0.7655 | Train AUC: 0.7815


INFO:sagemaker.experiments.run:The run (training-1-2023-03-12-54) under experiment (from-idea-to-prod-experiment-07-09-08-33) already exists. Loading it. Note: sagemaker.experiments.load_run is recommended to use when the desired run already exists.


Fit estimator with max_depth=5
[09:22:51] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Test AUC: 0.7721 | Train AUC: 0.8124


INFO:sagemaker.experiments.run:The run (training-2-2023-03-12-54) under experiment (from-idea-to-prod-experiment-07-09-08-33) already exists. Loading it. Note: sagemaker.experiments.load_run is recommended to use when the desired run already exists.


Fit estimator with max_depth=10
[09:22:52] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Test AUC: 0.7571 | Train AUC: 0.8592


INFO:sagemaker.experiments.run:The run (training-3-2023-03-12-54) under experiment (from-idea-to-prod-experiment-07-09-08-33) already exists. Loading it. Note: sagemaker.experiments.load_run is recommended to use when the desired run already exists.


Fit estimator with max_depth=15
[09:22:54] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Test AUC: 0.7627 | Train AUC: 0.8899


INFO:sagemaker.experiments.run:The run (training-4-2023-03-12-54) under experiment (from-idea-to-prod-experiment-07-09-08-33) already exists. Loading it. Note: sagemaker.experiments.load_run is recommended to use when the desired run already exists.


Fit estimator with max_depth=20
[09:22:57] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Test AUC: 0.7542 | Train AUC: 0.8966


## Explore experiment runs with Studio UX
You can see all logged metrics, parameters, and artifacts in Studio UX in **SageMaker Home** > **Experiments** widget.

For example, select your experiment:

![](img/experiment-and-runs.png)

In the experiment list, select the experiment to display a list of the runs in the experiment:

![](img/runs.png)

You can select runs you would like to analyse and click **Analyze**. A new window with selected runs opens:

![](img/run-analyze.png)

## Use experiment analytics
You can use the [analytics features](https://sagemaker.readthedocs.io/en/stable/api/training/analytics.html#analytics) of the Experiment SDK to query and compare the runs and identify the best model produced by your experiments.

Refer to these [notebooks](https://github.com/aws/amazon-sagemaker-examples/tree/main/sagemaker-experiments) for hands-on examples.

## Optional: Run the notebook as a SageMaker job
Sometimes there are scenarious in which you might want to run your notebooks as a non-interactive, scheduled jobs. Studio provides fast and simple tools built from the existing Amazon EventBridge, SageMaker Training and SageMaker Pipelines services to help you schedule your notebook jobs interactively. You don’t have to craft your own custom solution or enlist features from other services that may require additional overhead in time and costs to deploy.

You can run your notebook as a SageMaker job on-demand on based on any schedule you choose. You can also run multiple notebooks in parallel, and parametrize cells in your notebooks.

### Adapt the notebook to run headlessly
A headless notebook runs in a shell outside of the Studio environment. Therefore, your code in the notebook cannot depend on or access the Studio local storage, environment variables, or Python store. You must accordingly change any code which uses the local Studio environment.

### How to run
Follow the instructions in [Notebook-based Workflows](https://docs.aws.amazon.com/sagemaker/latest/dg/notebook-auto-run.html) in the Developer Guide to run this notebook in non-interactive mode as a SageMaker job:
1. [Configure](https://docs.aws.amazon.com/sagemaker/latest/dg/scheduled-notebook-policies.html) the trust policy and additional IAM permissions for the Studio execution role. If you run this notebook in the domain in the AWS-preprovisioned account, the required permissions are automatically deployed
2. Provide the parameters
3. Run the notebook on-demand or schedule a job
4. Explore the results

In [28]:
# output the name of the S3 bucket used by SageMaker
print(bucket_name)

sagemaker-us-east-1-906545278380


To parameterize your notebook, you set a tag `parameters` on a single cell in your notebook that marks it as the "parameter cell". SageMaker notebook execution will insert a cell directly after that at runtime with the parameter values you set when starting the job.

Since the notebook cannot access the Studio EFS volume, you must [set parameters that you pass to the notebook](https://docs.aws.amazon.com/sagemaker/latest/dg/notebook-auto-run-troubleshoot-override.html) to the following values:

```
file_source = S3
input_path = /opt/ml/input/data/sagemaker_headless_execution 
output_path = /opt/ml/output/data
bucket_name = SET TO YOUR SAGEMAKER BUCKET NAME
bucket_prefix = from-idea-to-prod/xgboost
```

In [26]:
# If running interactively, upload data to S3 to have it here for a headless run
if file_source == 'EFS':
    input_s3_url = session.upload_data(
        path=os.path.join(input_path, file_name),
        bucket=bucket_name,
        key_prefix=f"{bucket_prefix}/input"
    )
    
    print(input_s3_url)

s3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/input/bank-additional-full.csv


## Continue with the step 2
open the step 2 [notebook](02-sagemaker-containers.ipynb).

## Further development ideas for your real-world projects
- Try different models, for example some of the [SageMaker built-in algorithms](https://docs.aws.amazon.com/sagemaker/latest/dg/algos.html), such as [CatBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/catboost.html), [AutoGluon-Tabular](https://docs.aws.amazon.com/sagemaker/latest/dg/autogluon-tabular.html), or [Linear Learner Algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner.html)
- Try [SageMaker Autopilot](https://aws.amazon.com/sagemaker/autopilot/) to automatically explore different solutions to find the best model. Refer to this hands-on tutorial: [Automatically Create Machine Learning Models](https://aws.amazon.com/getting-started/hands-on/machine-learning-tutorial-automatically-create-models/)
- Implement batch inference using [SageMaker Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html)

## Additional resources
- [Build and Train a Machine Learning Model Locally](https://aws.amazon.com/getting-started/hands-on/machine-learning-tutorial-build-model-locally/)
- [Amazon SageMaker XGBoost algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html)
- [Automatically Create Machine Learning Models](https://aws.amazon.com/getting-started/hands-on/machine-learning-tutorial-automatically-create-models/)
- [Operationalize your Amazon SageMaker Studio notebooks as scheduled notebook jobs](https://aws.amazon.com/blogs/machine-learning/operationalize-your-amazon-sagemaker-studio-notebooks-as-scheduled-notebook-jobs/)
- [Dataset transformations](https://scikit-learn.org/stable/data_transforms.html)
- [Extracting, transforming and selecting features](https://spark.apache.org/docs/latest/ml-features.html)

# Shutdown kernel

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>